### Problem 1 - Matrix Multiplication

In [ ]:
import numpy as np
import timeit
import cython
%matplotlib ipympl

In [ ]:
def matMul(matA, matB):
    m, n = matA.shape
    n, p = matB.shape

    productMatrix = np.zeros((m, p))
    for i in range(m):
        for j in range(n):
            for k in range(p):
                productMatrix[i][j] += matA[i][k] * matB[k][j]
    
    return productMatrix

In [ ]:
A = np.random.random((10, 10))
B = np.random.random((10, 10))
timeMatMul = %timeit -o matMul(A, B)

It took 502 µs ± 8.35 µs to perform Matrix Multiplication of two $10\times10$ matrices during one test consisting of 7 runs of 1000 loops each

In [ ]:
timeNumPy = %timeit -o A@B

Multiplication the same two $10\times10$ matrices using A@B is taking 1.03 µs ± 13.6 ns during one test consisting of 7 runs of 1,000,000 loops each

#### Calculating FLOPS
Calculating the number of Floating Point Operations per Second (FLOPS) based on the total number of operations(multiplications $\times$ 2, assuming each multiplication is actually two floating point operations) and the time taken for numpy's matmul and the naive matrix multiplication algorithm to calculate the product of two $10\times10$ matrices.

In [ ]:
totalNoOfMultiplications = 10 * 10 * 10              
totalFLOPs = 2 * totalNoOfMultiplications            
matMulFLOPS = totalFLOPs / timeMatMul.average     
npFLOPS = totalFLOPs / timeNumPy.average     
matMulGFLOPS = matMulFLOPS / 1e9                            
npGFLOPS = npFLOPS / 1e9                    
print(f"The calculated number of GLOPS based on the Naive approach to Matrix Multiplication : {matMulGFLOPS:.3f} GFLOPS")          
print(f"The calculated number of GLOPS based on NumPy's matmul() function : {npGFLOPS:.3f} GFLOPS")

The calculated GFLOPS are 0.004 GFLOPS from the basic matrix multiplication caluclation and 1.935 GFLOPS using `NumPy.matmul()`.

### Step 2

Using the command `wmic cpu list /format:list `, that is Windows equivalent of the `lscpu` command on Linux, displayed that the maximum Clock Speed of the processor in my laptop is 4,700MHz, which is 4.7GHz, and it has 20 cores.
Based on this, and assuming that my CPU can perform either one addition or multiplication per clock cycle, the maximum FLOPS that may be achievable using a single processor core can be estimated as follows:

Number of Operations per second = $4.7\times10^9$. Time taken for a single operation = $\frac {1}{4.7\times10^9}\sec = 2.127\times10^{-10}\sec$ = 212.7 ps. Assuming the total number of floating point operations is twice the number of multiplications, the maximum number of floating point operations can be performed if all operations are multiplications and the number of operations that can be performed increases by a factor of $2$.

$\therefore$ The maximum FLOPS that may be achievable using a single processor core = Maximum possible number of Operations per second $\times 2 = 4.7\times10^9\times 2 = 9.4\times10^9$.

Comparing this with the GFLOPS calculated in Step 1 shows that some other processes are being performed in parallel with the matrix multiplication which causes the number of FLOPS to go down, roughly cut in $\frac {1}{4.86} ^{th}$ for `NumPy.matmul()`.

### Step 3


In [ ]:
numOfDoubles = 4 # Goes upto 10*2**3 = 80 x 80 matrices

matMulTimes = []
npMulTimes = []
matrixSize = [10*(2**i) for i in range(numOfDoubles)] # Defined to give the X Axis ticks in the plot

for i in range(numOfDoubles):
    A = np.random.random((10*(2**i), 10*(2**i)))
    B = np.random.random((10*(2**i), 10*(2**i)))

    time1 = %timeit -r 7 -n 100 -o matMul(A, B)
    time2 = %timeit -r 7 -n 100 -o  A@B

    matMulTimes.append(np.log(time1.average))
    npMulTimes.append(np.log(time2.average))


In [ ]:
import matplotlib.pyplot as plt

plt.plot(matrixSize, matMulTimes, "yo-", label = "Matrix Multiplication")
plt.plot(matrixSize, npMulTimes, "go-", label = "NumPy")
plt.xlabel("Order of the Matrix")
plt.ylabel("Log of the time taken in seconds")
plt.xticks(matrixSize)
plt.legend()

plt.show()

#### Times taken during a particular test of Matrix Multiplication of matrices of doubling in size.
The first time is the time taken for the basic matrix multiplication function and the second time is the time taken for numpy's matmul function

#### Matrix size is 10 x 10

475 µs ± 18.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

1.16 µs ± 150 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)

#### Matrix size is 20 x 20

3.9 ms ± 375 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

1.93 µs ± 161 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

#### Matrix size is 40 x 40

37 ms ± 1.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

5.64 µs ± 399 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)

#### Matrix size is 80 x 80

297 ms ± 9.34 ms per loop (mean ± std. dev. of 7 runs, 10 loop each)

447 µs ± 46.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

#### Matrix size is 160 x 160

2.23 s ± 53.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

544 µs ± 66.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

#### Matrix size is 320 x 320

18 s ± 670 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

629 µs ± 27.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

#### Matrix size is 640 x 640

2min 22s ± 5.91 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

3.77 ms ± 90.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

This shows that the increase in the time taken for `NumPy.matmul()` to run for higher and higher order matrices increases not too drastically compared to the naive implementation. The logrithm of the time taken for `np.matmul()` to compute the products of higher and higher order matrices increases roughly linearly with a slope of approximately 0.77 ${\log(seconds)}$/(1 increase in the order).

### Step 4 - Cython


Loading Cython and converting the Matrix Multipication function in step 1 to a function in Cython, and then finding the time taken for it to calculate the product of two $50\times50$ matrices.

In [ ]:
%load_ext Cython

In [ ]:
%%cython --annotate

import cython
import numpy as np

def matMul(matA, matB):
    m, n = matA.shape
    n, p = matB.shape

    productMatrix = np.zeros((m, p))
    for i in range(m):
        for j in range(n):
            for k in range(p):
                productMatrix[i][j] += matA[i][k] * matB[k][j]
    
    return productMatrix

In [ ]:
A = np.random.random((50, 50))
B = np.random.random((50, 50))
%timeit -n 1000 -r 3 matMul(A, B)

On converting the matrix multiplication function to cython, multiplying two matrices of the size 50 $\times$ 50 took 90.1 ms ± 1.78 ms per loop during on test consisting of 7 runs, 1000 loops each, when run on the Jupyter server.

### Step 5 - Transformations

Now a series of different and non-overlapping transformations will be performed on the matrix multiplication function's code making it closer to C (or Cython). Upon each transformation, it is timed and the improvements with respect to the non-transformed code are noted and compared. All of the transformed functions have been 

1. The first transformation is to declare the variables `m, n, p, i, j, k` to be of type `int`.

In [ ]:
%%cython --annotate

import cython
import numpy as np

def cIntMatMul(matA, matB):
    cdef int m, n, p, i, j, k
    m, n = matA.shape
    n, p = matB.shape

    productMatrix = np.zeros((m, p))
    for i in range(m):
        for j in range(n):
            for k in range(p):
                productMatrix[i][j] += matA[i][k] * matB[k][j]
    
    return productMatrix

2. The $2^{nd}$ transformation is to use the decorator `@cython.boundscheck(False)` to set `cython.boundscheck` to `False`.

In [ ]:
%%cython --annotate

import cython
import numpy as np

@cython.boundscheck(False)
def cBoundsCheckMatMul(matA, matB):
    m, n = matA.shape
    n, p = matB.shape

    productMatrix = np.zeros((m, p))
    for i in range(m):
        for j in range(n):
            for k in range(p):
                productMatrix[i][j] += matA[i][k] * matB[k][j]
    
    return productMatrix

3. The $3^{rd}$ transformation is to declare the input variables to be of type `double[:, :]`.

In [ ]:
%%cython --annotate

import cython
import numpy as np

def cDoublesMatMul(double[:,:] matA, double[:,:] matB):
    m, n = matA.shape[0], matA.shape[1]
    p = matB.shape[1]

    productMatrix = np.zeros((m, p))
    for i in range(m):
        for j in range(n):
            for k in range(p):
                productMatrix[i][j] += matA[i][k] * matB[k][j]
    
    return productMatrix

4. The $4^{th}$ transformation is to declare the input variables and the `productMatrix` to be of type `double[:, :]` and to pass the `productMatrix` as an argument to the matrix multiplication function making sure that `productMatrix` is initialized to a zero array before calling.

In [ ]:
%%cython --annotate

import cython
import numpy as np

def cDoublesResMatMul(double[:,:] matA, double[:,:] matB, double[:,:] productMatrix):
    m, n = matA.shape[0], matA.shape[1]
    p = matB.shape[1]

    for i in range(m):
        for j in range(n):
            for k in range(p):
                productMatrix[i][j] += matA[i][k] * matB[k][j]
    
    return productMatrix

1. The $5^{th}$ transformation is to declare the input variables to be of type `float[:, :]` .

In [ ]:
%%cython --annotate

import cython
import numpy as np

def cFloatsMatMul(float[:,:] matA, float[:,:] matB):
    m, n = matA.shape[0], matA.shape[1]
    p = matB.shape[1]

    productMatrix = np.zeros((m, p))
    for i in range(m):
        for j in range(n):
            for k in range(p):
                productMatrix[i][j] += matA[i][k] * matB[k][j]
    
    return productMatrix

6. The $6^{th}$ transformation is to declare the input variables and the `productMatrix` to be of type `float[:, :]` and to pass the `productMatrix` as an argument to the matrix multiplication function making sure that `productMatrix` is initialized to a zero array before calling.

In [ ]:
%%cython --annotate

import cython
import numpy as np

def cFloatsResMatMul(float[:,:] matA, float[:,:] matB, float[:,:] productMatrix):
    m, n = matA.shape[0], matA.shape[1]
    p = matB.shape[1]

    for i in range(m):
        for j in range(n):
            for k in range(p):
                productMatrix[i][j] += matA[i][k] * matB[k][j]
    
    return productMatrix

7. The $7^{th}$ transformation is to declare the input variables and the `productMatrix` to be of type `float[:, :]` and to pass the `productMatrix` as an argument to the matrix multiplication function making sure that `productMatrix` is initialized to a zero array before calling, to use the decorator `@cython.boundscheck(False)`, and to declare the variables `m, n, p, i, j, k` to be of type `int`.

The 3 kinds of transformations are used together (`float` was used instead of `double` because it seemed to give a slightly better result).

In [ ]:
%%cython --annotate

import cython
import numpy as np

@cython.boundscheck(False)
def cMatMul(float[:,:] matA, float[:,:] matB, float[:,:] productMatrix):
    cdef int m, n, p, i, j, k
    m, n = matA.shape[0], matA.shape[1]
    p = matB.shape[1]

    for i in range(m):
        for j in range(n):
            for k in range(p):
                productMatrix[i][j] += matA[i][k] * matB[k][j]
    
    return productMatrix

Running each of the transformed functions and timing them.

In [ ]:
A = np.random.random((20, 20))
B = np.random.random((20, 20))

%timeit -r 7 -n 1000 cIntMatMul(A, B)
%timeit -r 7 -n 1000 cBoundsCheckMatMul(A, B)

productMat = np.zeros((20,20))
%timeit -r 7 -n 1000 cDoublesMatMul(A, B)
%timeit -r 7 -n 1000 cDoublesResMatMul(A, B, productMat)

productMat = np.zeros((20,20))
%timeit -r 7 -n 1000 cFloatsMatMul(A.astype(np.float32), B.astype(np.float32))
%timeit -r 7 -n 1000 cFloatsResMatMul(A.astype(np.float32), B.astype(np.float32), productMat.astype(np.float32))

productMat = np.zeros((20,20))
%timeit -r 7 -n 1000 cMatMul(A.astype(np.float32), B.astype(np.float32), productMat.astype(np.float32))

For reference it took 16.4 ms $\pm$ 3.4 ms to perform Matrix Multiplication of two $20\times20$ matrices during one test consisting of 7 runs of 1000 loops each, on the Jupyter Server.
All of the timings have been done by running the function for 7 runs of 1000 loops each on the Jupyter Server.

1. Defining `m, n, p, i, j, k` to be of type `int` brought the time taken for the Multiplication of two $20\times20$ matrices down to **8.63 ms ± 2.34 ms**
   
2. Check Bounds brought the time taken for the Multiplication of two $20\times20$ matrices down to **8.56 ms ± 1.22 ms**

3. Defining `A, B` to be of type `double` brought the time taken for the Multiplication of two $20\times20$ matrices down to **2.85 ms ± 908 µs**

4. Defining `A, B, productMatrix` to be of type `double` and passing the Product Matrix as an Argument brought the time taken for the Multiplication of two $20\times20$ matrices down to **24.1 µs ± 1.92 µs**
   
5. Defining Floats brought the time taken for the Multiplication of two $20\times20$ matrices down to **1.74 ms ± 34.5 µs**

6.  Defining Floats and Product Matrix as an Argument brought the time taken for the Multiplication of two $20\times20$ matrices down to **25.1 µs ± 920 ns**
   
7.  Defining Ints, Floats and Product Matrix as an Argument brought the time taken for the Multiplication of two $20\times20$ matrices down to **12 µs ± 1.77 µs**

Running NumPy's matmul function to perform Matrix Multiplication of two $20\times20$ matrices took **7.43 µs ± 3.57 µs** per loop

### Observations and Conclusion
Based on the tests done on the Jupyter Server, the single change that resulted in the best improvement is the transformation in which the input matrices, along with the product matrix, which is being passed as an argument, have been declared to be of type `float`. This transformation gave the best average speed out of all of the transformations which is **25.1 µs ± 920 ns**. 

**Reason**

Python uses double-precision floating-point numbers by default, while Cython uses single-precision floating-point numbers by default. This means that when you pass a NumPy array of type float to a Cython function, it will be converted to single-precision floating-point.By explicitly declaring the input matrices and the product matrix to be of type float, you are ensuring that they are all using the same data type. This avoids the need for any conversions, which can improve performance. In addition, Cython is able to generate more optimized code for single-precision floating-point operations. This is because single-precision floating-point operations are typically faster than double-precision floating-point operations.


Using the 3 different kinds of transformations resulted in a time of **12 µs ± 1.77 µs** which is the best overall. On comparing the two previous transformations with the time taken for NumPy's matmul it can be observed that `numpy.matmul` is barely faster than the function that had all of the possible transformations given and is roughly $3\times$ faster than the best single transformed cython function. Thus one can conclude that `numpy.matmul()` is heavily optimized, but the results from Cython and NumPy are roughly the same and most of the times `Cython` code outperforms `NumPy` code, often by an order of magnitude. This is because Cython can generate optimized machine code for your code, which can be much faster than the interpreted Python code that NumPy uses, it can avoid the overhead of the Python wrapper that is used to call NumPy functions, and it can take advantage of the specific data types.